In [ ]:
!pip install insightface
!pip install -U mxnet
!pip install keras_applications

  Created wheel for insightface: filename=insightface-0.1.5-cp36-none-any.whl size=16766 sha256=37c2c28faac0f1431ad09cea3925a7a86b005847a2a555bc369d139f4a796cb1
  Stored in directory: /root/.cache/pip/wheels/11/b8/83/2bac12a833d66d58aa9705f5e7d94d2e9055f16f4698af0df7
Successfully built insightface
     |████████████████████████████████| 55.0MB 81kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 51kB 2.6MB/s 


In [ ]:
import insightface
import urllib
import urllib.request
import cv2
import mxnet as mx
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from keras.preprocessing import image

In [ ]:
model = insightface.model_zoo.get_model('arcface_r100_v1', pretrained=True, include_top=False)

Model file is not found. Downloading.


100%|██████████| 237710/237710 [00:20<00:00, 11851.09KB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp -r "/content/drive/My Drive/test" "/contest/data"

# Caucasian pairs


In [ ]:
# Считаем уже отобранные 6000 пар по cosin similarity
Caucasian_pais = '/content/drive/My Drive/test/txts/Caucasian/Caucasian_pairs.txt'
pairs = pd.read_csv(Caucasian_pais, header=None)
pairs = pairs[0]
rows = []
for pair in pairs:
  if len(pair.split('\t'))>3:
      row = pair.split('\t')
      rows.append([row[0],row[2],int(row[1]), int(row[3]), 0])
  else:
      row = pair.split('\t')
      rows.append([row[0],row[0], int(row[1]), int(row[2]), 1])    
data_pair = pd.DataFrame(rows, columns=['person_l','person_r','number_l','number_r','same'])
data_pair.head()

,person_l,person_r,number_l,number_r,same
0,m.027n6t8,m.027n6t8,3,2,1
1,m.016lb3,m.016lb3,4,2,1
2,m.02_4qt,m.02_4qt,3,2,1
3,m.09g8fj,m.09g8fj,3,1,1
4,m.01q3xr,m.01q3xr,1,3,1


In [ ]:
ROOT_FOLDER = '/content/drive/My Drive/test/data/Caucasian/'
dataset = []
for path in glob.iglob(os.path.join(ROOT_FOLDER, "**", "*.jpg")):
    person = path.split("/")[-2]
    dataset.append({"person":person, "path": path})
print(len(dataset))
dataset = pd.DataFrame(dataset)
dataset['number'] = dataset.path.apply(lambda x: int(x.split('_')[-1].split('.')[0]))
dataset.head()

10196


,person,path,number
0,m.027d4p,/content/drive/My Drive/test/data/Caucasian/m....,3
1,m.027d4p,/content/drive/My Drive/test/data/Caucasian/m....,2
2,m.027d4p,/content/drive/My Drive/test/data/Caucasian/m....,1
3,m.09y58j,/content/drive/My Drive/test/data/Caucasian/m....,1
4,m.09y58j,/content/drive/My Drive/test/data/Caucasian/m....,3


In [ ]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [ ]:
%time
sim = []
model.prepare(ctx_id = -1)
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):

  img = image.load_img(data_pair.path_l[idx], target_size=(112, 112))
  x = image.img_to_array(img)
  preds_x = model.get_embedding(x)

  img = image.load_img(data_pair.path_r[idx], target_size=(112, 112))
  y = image.img_to_array(img)
  preds_y = model.get_embedding(y)

  data_pair.loc[idx, 'sim'] = cosine_similarity(preds_x, preds_y)[0][0]
result = pd.DataFrame({'target':data_pair.same, 'score': data_pair.sim})

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


100%|██████████| 6000/6000 [3:12:13<00:00,  1.92s/it]


In [ ]:
result = result.sort_values('score', ascending=False)
roc_auc_score(result.target, result.score)

0.9602392222222222

# African pairs

In [ ]:
# Считаем уже отобранные 6000 пар по cosin similarity
African_pais = '/content/drive/My Drive/test/txts/African/African_pairs.txt'
pairs = pd.read_csv(African_pais, header=None)
pairs = pairs[0]
rows = []
for pair in pairs:
  if len(pair.split('\t'))>3:
      row = pair.split('\t')
      rows.append([row[0],row[2],int(row[1]), int(row[3]), 0])
  else:
      row = pair.split('\t')
      rows.append([row[0],row[0], int(row[1]), int(row[2]), 1])    
data_pair = pd.DataFrame(rows, columns=['person_l','person_r','number_l','number_r','same'])
data_pair.head()

,person_l,person_r,number_l,number_r,same
0,m.0g5ty_,m.0g5ty_,1,2,1
1,m.05q5bqf,m.05q5bqf,2,3,1
2,m.03yfnz5,m.03yfnz5,2,1,1
3,m.03npynb,m.03npynb,2,3,1
4,m.0h1f_hh,m.0h1f_hh,1,3,1


In [ ]:
ROOT_FOLDER = '/content/drive/My Drive/test/data/African/'
dataset = []
for path in glob.iglob(os.path.join(ROOT_FOLDER, "**", "*.jpg")):
    person = path.split("/")[-2]
    dataset.append({"person":person, "path": path})
print(len(dataset))
dataset = pd.DataFrame(dataset)
dataset['number'] = dataset.path.apply(lambda x: int(x.split('_')[-1].split('.')[0]))
dataset['key'] =1
dataset.head()

10415


,person,path,number,key
0,m.05jwk9,/content/drive/My Drive/test/data/African/m.05...,3,1
1,m.05jwk9,/content/drive/My Drive/test/data/African/m.05...,5,1
2,m.05jwk9,/content/drive/My Drive/test/data/African/m.05...,1,1
3,m.05jwk9,/content/drive/My Drive/test/data/African/m.05...,2,1
4,m.05jwk9,/content/drive/My Drive/test/data/African/m.05...,4,1


In [ ]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [ ]:
%time
sim = []
model.prepare(ctx_id = -1)
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):

  img = image.load_img(data_pair.path_l[idx], target_size=(112, 112))
  x = image.img_to_array(img)
  preds_x = model.get_embedding(x)

  img = image.load_img(data_pair.path_r[idx], target_size=(112, 112))
  y = image.img_to_array(img)
  preds_y = model.get_embedding(y)

  data_pair.loc[idx, 'sim'] = cosine_similarity(preds_x, preds_y)[0][0]
result = pd.DataFrame({'target':data_pair.same, 'score': data_pair.sim})

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


100%|██████████| 6000/6000 [3:01:20<00:00,  1.81s/it]


In [ ]:
result = result.sort_values('score', ascending=False)
roc_auc_score(result.target, result.score)

0.9149914444444444

# Indian Pairs

In [15]:
# Считаем уже отобранные 6000 пар по cosin similarity
Indian_pais = '/content/drive/My Drive/test/txts/Indian/Indian_pairs.txt'
pairs = pd.read_csv(Indian_pais, header=None)
pairs = pairs[0]
rows = []
for pair in pairs:
  if len(pair.split('\t'))>3:
      row = pair.split('\t')
      rows.append([row[0],row[2],int(row[1]), int(row[3]), 0])
  else:
      row = pair.split('\t')
      rows.append([row[0],row[0], int(row[1]), int(row[2]), 1])    
data_pair = pd.DataFrame(rows, columns=['person_l','person_r','number_l','number_r','same'])
data_pair.head()

,person_l,person_r,number_l,number_r,same
0,m.04ld_9f,m.04ld_9f,1,3,1
1,m.02qv6d7,m.02qv6d7,2,1,1
2,m.0j9ncvv,m.0j9ncvv,3,1,1
3,m.034kdq,m.034kdq,1,2,1
4,m.04qp06,m.04qp06,1,4,1


In [ ]:
ROOT_FOLDER = '/content/drive/My Drive/test/data/Indian/'
dataset = []
for path in glob.iglob(os.path.join(ROOT_FOLDER, "**", "*.jpg")):
    person = path.split("/")[-2]
    dataset.append({"person":person, "path": path})
print(len(dataset))
dataset = pd.DataFrame(dataset)
dataset['number'] = dataset.path.apply(lambda x: int(x.split('_')[-1].split('.')[0]))
dataset['key'] =1
dataset.head()

10308


,person,path,number,key
0,m.0g54n2,/content/drive/My Drive/test/data/Indian/m.0g5...,1,1
1,m.0g54n2,/content/drive/My Drive/test/data/Indian/m.0g5...,2,1
2,m.0g54n2,/content/drive/My Drive/test/data/Indian/m.0g5...,3,1
3,m.0g54n2,/content/drive/My Drive/test/data/Indian/m.0g5...,4,1
4,m.038j0b,/content/drive/My Drive/test/data/Indian/m.038...,3,1


In [ ]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [14]:
%time
sim = []
model.prepare(ctx_id = -1)
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):

  img = image.load_img(data_pair.path_l[idx], target_size=(112, 112))
  x = image.img_to_array(img)
  preds_x = model.get_embedding(x)

  img = image.load_img(data_pair.path_r[idx], target_size=(112, 112))
  y = image.img_to_array(img)
  preds_y = model.get_embedding(y)

  data_pair.loc[idx, 'sim'] = cosine_similarity(preds_x, preds_y)[0][0]
result = pd.DataFrame({'target':data_pair.same, 'score': data_pair.sim})

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


100%|██████████| 6000/6000 [3:01:02<00:00,  1.81s/it]


In [16]:
result = result.sort_values('score', ascending=False)
roc_auc_score(result.target, result.score)

0.9360234444444444

# Asian pairs

In [ ]:
# Считаем уже отобранные 6000 пар по cosin similarity
Asian_pais = '/content/drive/My Drive/test/txts/Asian/Asian_pairs.txt'
pairs = pd.read_csv(Asian_pais, header=None)
pairs = pairs[0]
rows = []
for pair in pairs:
  if len(pair.split('\t'))>3:
      row = pair.split('\t')
      rows.append([row[0],row[2],int(row[1]), int(row[3]), 0])
  else:
      row = pair.split('\t')
      rows.append([row[0],row[0], int(row[1]), int(row[2]), 1])    
data_pair = pd.DataFrame(rows, columns=['person_l','person_r','number_l','number_r','same'])
data_pair.head()

In [18]:
ROOT_FOLDER = '/content/drive/My Drive/test/data/Asian/'
dataset = []
for path in glob.iglob(os.path.join(ROOT_FOLDER, "**", "*.jpg")):
    person = path.split("/")[-2]
    dataset.append({"person":person, "path": path})
print(len(dataset))
dataset = pd.DataFrame(dataset)
dataset['number'] = dataset.path.apply(lambda x: int(x.split('_')[-1].split('.')[0]))
dataset['key'] =1
dataset.head()

9688


,person,path,number,key
0,m.04xvn3,/content/drive/My Drive/test/data/Asian/m.04xv...,5,1
1,m.04xvn3,/content/drive/My Drive/test/data/Asian/m.04xv...,2,1
2,m.04xvn3,/content/drive/My Drive/test/data/Asian/m.04xv...,6,1
3,m.04xvn3,/content/drive/My Drive/test/data/Asian/m.04xv...,3,1
4,m.04xvn3,/content/drive/My Drive/test/data/Asian/m.04xv...,1,1


In [19]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [20]:
%time
sim = []
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):

  img = image.load_img(data_pair.path_l[idx], target_size=(112, 112))
  x = image.img_to_array(img)
  preds_x = model.get_embedding(x)

  img = image.load_img(data_pair.path_r[idx], target_size=(112, 112))
  y = image.img_to_array(img)
  preds_y = model.get_embedding(y)

  data_pair.loc[idx, 'sim'] = cosine_similarity(preds_x, preds_y)[0][0]
result = pd.DataFrame({'target': data_pair.same, 'score': data_pair.sim})

  0%|          | 0/6000 [00:00<?, ?it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


100%|██████████| 6000/6000 [3:03:21<00:00,  1.83s/it]


In [21]:
result = result.sort_values('score', ascending=False)
roc_auc_score(result.target, result.score)

0.9179011666666665